In [ ]:
from ipycanvas import Canvas
import math

In [ ]:
def _get_queens(board):
    if isinstance(board, tuple):
        board, n = board
        return board, n
    elif isinstance(board[0], list):
        queens = [(r, c) 
                  for r, row in enumerate(board)
                  for c, cell in enumerate(row)
                  if cell == 'Q']
    else:
        queens = [(r, c) for r, c in enumerate(board)]
    return queens, len(board)

def find_conflicts(board):
    queens, _ = _get_queens(board)
    return find_conflicts_impl(queens)
    
def find_conflicts_impl(queens):
    conflicts = set()
    def add_if_conflict(d, k, v):
        if k in d:
            conflicts.add(d[k])
            conflicts.add(v)
        else:
            d[k] = v
        
    rows = dict()
    cols = dict()
    ldiags = dict()
    rdiags = dict()
    
    for coord in queens:
        r, c = coord
        add_if_conflict(rows, r, coord)
        add_if_conflict(cols, c, coord)
            
        ldiag = c - r
        add_if_conflict(ldiags, ldiag, coord)
        rdiag = c + r
        add_if_conflict(rdiags, rdiag, coord)
    
    return conflicts

In [ ]:
def _offset(grid_px, debug):
    return 2 * grid_px if debug else 0

def draw_nqueens_impl(queens, n, grid_px, debug, show_conflicts):
    width = n * grid_px + _offset(grid_px, debug)
    canvas = Canvas(width=width, height=width)
    draw_nqueens_canvas_impl(canvas, queens, n, grid_px, debug, show_conflicts)
    return canvas

def draw_nqueens_canvas_impl(canvas, queens, n, grid_px, debug, show_conflicts):
    offset_px = _offset(grid_px, debug)
    center_px = grid_px / 2
    
    def is_black(r, c):
        '''Bottom left must be black'''
        row_first_black = (n - r) % 2 == 0
        is_even_col = c % 2 == 0
        return row_first_black ^ is_even_col
    
    def to_px(r, c, center=False, offset=True):
        x_px = c * grid_px
        y_px = r * grid_px
        if center:
            x_px += center_px
            y_px += center_px
        if offset:
            x_px += offset_px
            y_px += offset_px
        return x_px, y_px

    
    black = [
        to_px(r, c)
        for r in range(n)
        for c in range(n)
        if is_black(r, c)
    ]
    
    # Draw checkers.
    canvas.fill_style = 'black'
    x_pxs, y_pxs = zip(*black)
    x_pxs, y_pxs = list(x_pxs), list(y_pxs)
    canvas.fill_rects(x_pxs, y_pxs, grid_px)
    
    if show_conflicts:
        conflicts = find_conflicts_impl(queens)
        if len(conflicts):
            canvas.fill_style = 'red'
            pxs = [to_px(*coord) for coord in conflicts]
            x_pxs, y_pxs = zip(*pxs)
            x_pxs, y_pxs = list(x_pxs), list(y_pxs)
            canvas.fill_rects(x_pxs, y_pxs, grid_px)

    # Draw queens
    canvas.text_align = 'center'
    canvas.text_baseline = 'middle'
    canvas.font = f'{grid_px}px Courier'
    for r, c in queens:
        if show_conflicts and (r, c) in conflicts:
            canvas.fill_style = 'black'
        elif is_black(r, c):
            canvas.fill_style = 'white'
        else:
            canvas.fill_style = 'black'
        x_px, y_px = to_px(r, c)
        canvas.fill_text('♕', x_px + grid_px / 2, y_px + grid_px / 2)
        
    # Draw debug information.
    if debug:
        canvas.fill_style = "black"
        canvas.font = f"{grid_px}px Courier"
        canvas.text_align = "center"
        canvas.text_baseline = "middle"
        for r in range(n):
            x_px, y_px = to_px(r + 2, 1, offset=False, center=True)
            canvas.fill_text(str(r), x_px, y_px)

        for c in range(n):
            x_px, y_px = to_px(1, c + 2, offset=False, center=True)
            canvas.fill_text(str(c), x_px, y_px)

        # X label
        x_px = offset_px + n * grid_px / 2
        y_px = center_px
        canvas.fill_text("Column", x_px, y_px)

        # Y label
        x_px = center_px
        y_px = offset_px + n * grid_px / 2
        canvas.translate(x_px, y_px)
        canvas.rotate(math.radians(-90))
        canvas.fill_text("Row", 0, 0)
        canvas.rotate(math.radians(90))
        canvas.translate(-x_px, -y_px)
    
    return canvas

def draw_nqueens(board, grid_px=60, debug=True, show_conflicts=False):
    queens, n = _get_queens(board)
    return draw_nqueens_impl(queens, n, grid_px, debug, show_conflicts)

In [ ]:
draw_nqueens([1, 3, 0, 2, 7, 5, 3, 1])

board = [
    [' ', 'Q', ' ', ' '],
    ['Q', ' ', ' ', ' '],
    [' ', ' ', ' ', ' '],
    [' ', ' ', ' ', 'Q']
]
draw_nqueens(board, show_conflicts=True)

In [ ]:
board = [
    [' ', ' ', ' ', 'Q'],
    ['Q', ' ', ' ', ' '],
    [' ', ' ', ' ', ' '],
    [' ', 'Q', ' ', ' '],
]
print(find_conflicts(board))
draw_nqueens(board)

In [ ]:
queens = [(0, 1), (1, 2)]
draw_nqueens((queens, 4), show_conflicts=True)

In [ ]:
n = 8
board = [[' ' for _ in range(n)] for _ in range(n)]

def solve(board, r, n):
    for c in range(n):
        board[r][c] = 'Q'
        if not find_conflicts(board):
            if r == n - 1:
                return True
            
            if solve(board, r + 1, n):
                return True
        board[r][c] = ' '
        
    return False

solve(board, 0, n)

In [ ]:
draw_nqueens(board, grid_px=30)

In [ ]:
n = 16
board = []

def solve(board, r, n):
    for c in range(n):
        board.append((r, c))
        if not find_conflicts((board, n)):
            if r == n - 1:
                return True
            
            if solve(board, r + 1, n):
                return True
        board.pop()
    return False

solve(board, 0, n)

In [ ]:
draw_nqueens((board, n), grid_px=30)

In [ ]:
import random
random.seed(6)
n = 8
board = list(range(n))
random.shuffle(board)

actions = list()
for r1 in range(n):
    for r2 in range(r1):
        board[r1], board[r2] = board[r2], board[r1]
        actions.append((len(find_conflicts(board)), r1, r2))
        board[r1], board[r2] = board[r2], board[r1]
        
best = min(actions)
score, r1, r2 = best

print(best)
board[r1], board[r2] = board[r2], board[r1]

draw_nqueens(board, show_conflicts=True)